In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

import warnings
warnings.filterwarnings('ignore')

from flask import Flask
from flask import request
from flask import render_template
import mysql.connector
from mysql.connector import Error
import string

In [63]:
# Connect to MYSQL Database
connection = mysql.connector.connect(host='192.168.12.102',
            database='vonly_data_feed_us_staging',
            user='vonly-agent',
            password='a714fded-311c-4215-8b8b-5df4086e264b')

In [64]:
# for id_1 in v_id:
#     print("\n","Test Case: ",test_case)
#     #print(id_1)
#     test_case=test_case+1
# 444bd446-fff0-479a-b535-859c9a75a796
# 40f43d4d-206a-4430-8232-a7ea89b7fca0
# First SQL Query
query1 = """
    SELECT ti.platform,ti.title ,ti.vonly_asset_id source_vonly,ti.num,ti.portal_item_id ,
t.director,t.actors,t.name,t.writers,t.produced_by,t.distributed_by_parent,t.producers,t.runtime 
FROM tv_ids ti
LEFT OUTER JOIN sandbox.tvshows t ON ti.id = t.vonly_id AND ti.region = t.region
WHERE ti.vonly_asset_id = '043057d4-9f99-416c-8fc7-7b887ff01dd0';
 
"""

df1 = pd.read_sql_query(query1, connection)

# Second SQL Query
query2 = """
    SELECT tis.platform,tis.title des_title,tis.vonly_asset_id source_vonly,tis.num des_num_season,tis.portal_item_id source_portal_id, 
tie.platform,tie.title,tie.vonly_asset_id des_vonly,tie.num des_num_epi,tie.portal_item_id des_portal_id,t.director,t.actors,
t.name,t.writers,t.produced_by,t.distributed_by_parent,t.producers,t.runtime
FROM tv_id_mappings tim
LEFT OUTER JOIN tv_ids tis ON tim.season_vonly_id=tis.id 
LEFT OUTER JOIN tv_ids tie ON tim.episode_vonly_id=tie.id
LEFT OUTER JOIN sandbox.tvshows t ON tis.id=t.vonly_id AND tis.region=t.region
WHERE tis.vonly_asset_id IN ('043057d4-9f99-416c-8fc7-7b887ff01dd0')
ORDER BY 4,8;
"""

df2 = pd.read_sql_query(query2, connection)

# Close the database connection
connection.close()
 



In [14]:
df1.columns

Index(['platform', 'title', 'source_vonly', 'num', 'portal_item_id',
       'director', 'actors', 'name', 'writers', 'produced_by',
       'distributed_by_parent', 'producers', 'runtime'],
      dtype='object')

In [15]:
df2.columns

Index(['platform', 'des_title', 'source_vonly', 'des_num_season',
       'source_portal_id', 'platform', 'title', 'des_vonly', 'des_num_epi',
       'des_portal_id', 'director', 'actors', 'name', 'writers', 'produced_by',
       'distributed_by_parent', 'producers', 'runtime'],
      dtype='object')

In [16]:
df1.head(3)

,platform,title,source_vonly,num,portal_item_id,director,actors,name,writers,produced_by,distributed_by_parent,producers,runtime
0,Amazon Prime Video,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,B00SR30J0S,None,None,/DRIVE,None,NBC,Universal,None,None
1,AppleTVApp,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,955956572,J.F. Musial,None,/DRIVE,"Zack Klapman, Mike Spinelli",NBC Sports,Universal,,None
2,AppleTVApp,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,umc.cmc.5kgs40rh3n5yhc7gf054kvl3x,J.F. Musial,None,/DRIVE,"Zack Klapman, Mike Spinelli",NBC Sports,Universal,,None


In [81]:
df2.head(6)

,platform,des_title,source_vonly,des_num_season,source_portal_id,platform,title,des_vonly,des_num_epi,des_portal_id,director,actors,name,writers,produced_by,distributed_by_parent,producers,runtime
0,Amazon Prime Video,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,B00SR30J0S,Amazon Prime Video,Opposites Attract,17d7194a-499d-440a-9958-7d4af2f68c97,6,B00SR364SO,None,None,/DRIVE,None,NBC,Universal,None,None
1,Amazon Prime Video,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,B00SR30J0S,Amazon Prime Video,Opposites Attract,17d7194a-499d-440a-9958-7d4af2f68c97,6,B00SR38RFM,None,None,/DRIVE,None,NBC,Universal,None,None
2,AppleTVApp,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,umc.cmc.5kgs40rh3n5yhc7gf054kvl3x,AppleTVApp,Opposites Attract,17d7194a-499d-440a-9958-7d4af2f68c97,6,956761498,J.F. Musial,None,/DRIVE,"Zack Klapman, Mike Spinelli",NBC Sports,Universal,,None
3,iTunes,"/DRIVE, Season 1",043057d4-9f99-416c-8fc7-7b887ff01dd0,1,955956572,iTunes,Opposites Attract,17d7194a-499d-440a-9958-7d4af2f68c97,6,956761498,None,None,"/DRIVE, Season 1",None,"NBCUniversal Media, LLC",Universal,None,None
4,Amazon Prime Video,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,B00SR34X5U,Amazon Prime Video,Opposites Attract,17d7194a-499d-440a-9958-7d4af2f68c97,6,B00SR364SO,None,None,/DRIVE,None,NBC,Universal,None,None
5,Amazon Prime Video,/DRIVE,043057d4-9f99-416c-8fc7-7b887ff01dd0,1,B00SR34X5U,Amazon Prime Video,Opposites Attract,17d7194a-499d-440a-9958-7d4af2f68c97,6,B00SR38RFM,None,None,/DRIVE,None,NBC,Universal,None,None


# Two query Comparison

In [45]:
mapping_list = []
id_ = 0

for _, row1 in df1.iterrows():
    id_ = id_ + 1
    
    count = 0
    mapped = 0
    unmapped = 0
    
    matching_points = {
        'id': id_,
        'title': 0,
        'title_name':row1['title'],
        'title_name_1':row2['title'],
        'director': 0,
        'director_name':row1['director'],
        'director_name_1':row2['director'],
        'producers': 0,
        'producer_name':row1['producers'],
        'producer_name_1':row2['producers'],
        'portal_item_id': 0,
        'portal_item_id_':row1['portal_item_id'],
        'portal_item_id_1':row2['des_portal_id'],
        'writers': 0,
        'writer_name':row1['writers'],
        'writer_name_1':row2['writers'],
        'runtime': 0,
        'runtime_':row1['runtime'],
        'runtime_1':row2['runtime'],
        'count': count,
        'mapped': mapped,
        'unmapped': unmapped
    }

    for _, row2 in df2.iterrows():
        
        if not pd.isnull(row1['title']) and not pd.isnull(row2['title']) and row1['title'] == row2['title']:
            matching_points['title'] = 1

        if not pd.isnull(row1['director']) and not pd.isnull(row2['director']) and row1['director'] == row2['director']:
            matching_points['director'] = 1

        if not pd.isnull(row1['producers']) and not pd.isnull(row2['producers']) and row1['producers'] == row2['producers']:
            matching_points['producers'] = 1
            
        if not pd.isnull(row1['portal_item_id']) and not pd.isnull(row2['des_portal_id']) and row1['portal_item_id'] == row2['des_portal_id']:
            matching_points['portal_item_id'] = 1

        if not pd.isnull(row1['writers']) and not pd.isnull(row2['writers']) and row1['writers'] == row2['writers']:
            matching_points['writers'] = 1
            
        if not pd.isnull(row1['runtime']) and not pd.isnull(row2['runtime']) and row1['runtime'] == row2['runtime']:
            matching_points['runtime'] = 1
        
        count = matching_points['title'] + matching_points['director'] + matching_points['producers'] + matching_points['portal_item_id'] + matching_points['writers'] + matching_points['runtime']
        

    mapped = 1 if count >= 3 else 0
    unmapped = 1 if count < 3 else 0

    matching_points['count'] = count
    matching_points['mapped'] = mapped
    matching_points['unmapped'] = unmapped
    
    mapping_list.append(matching_points)

mapping_list



[{'id': 1,
  'title': 0,
  'title_name': '/DRIVE',
  'title_name_1': 'Best Made Cars In The World',
  'director': 0,
  'director_name': None,
  'director_name_1': None,
  'producers': 0,
  'producer_name': None,
  'producer_name_1': None,
  'portal_item_id': 0,
  'portal_item_id_': 'B00SR30J0S',
  'portal_item_id_1': 'bURVga03uI0',
  'writers': 0,
  'writer_name': None,
  'writer_name_1': None,
  'runtime': 0,
  'runtime_': None,
  'runtime_1': None,
  'count': 0,
  'mapped': 0,
  'unmapped': 1},
 {'id': 2,
  'title': 0,
  'title_name': '/DRIVE',
  'title_name_1': 'Best Made Cars In The World',
  'director': 1,
  'director_name': 'J.F. Musial',
  'director_name_1': None,
  'producers': 1,
  'producer_name': '',
  'producer_name_1': None,
  'portal_item_id': 0,
  'portal_item_id_': '955956572',
  'portal_item_id_1': 'bURVga03uI0',
  'writers': 1,
  'writer_name': 'Zack Klapman, Mike Spinelli',
  'writer_name_1': None,
  'runtime': 0,
  'runtime_': None,
  'runtime_1': None,
  'count': 3

#  Self Comparison

In [80]:
mapping_list = []
id_ = 0

for _, row1 in df2.iterrows():
    id_ += 1
    matching_points = {
        'id': id_,
        'title': 0,
        'title_name': row1['title'],
        'title_name_1': row2['title'],
    }

    for _, row2 in df2.iterrows():
        if row1['des_num_epi'] == row2['des_num_epi']:
            if row1['title'] == row2['title']:
                matching_points['title'] = 1
                break  # Break out of the loop once a match is found
#             else:
#                 print("Unmapped")
        else:
            matching_points['title'] = 0

    mapping_list.append(matching_points)

mapping_list


[{'id': 1,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Best Made Cars In The World'},
 {'id': 2,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 3,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 4,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 5,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 6,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 7,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 8,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 9,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_name_1': 'Opposites Attract'},
 {'id': 10,
  'title': 1,
  'title_name': 'Opposites Attract',
  'title_

In [84]:
df = pd.DataFrame(mapping_list)

In [85]:
df.to_csv("Mapping.csv")